Задача программы - найти нужные файлы и страницы в этих файлах по ключевым словам:
т.е. мы хотим автоматически найти в большой куче файлов нужные нам и отсеять их в отдельную таблицу. 

input:
1) Путь к zip архиву
2) Пароль от архива (в случае отсутствия пустая строка)
3) Строку (NAME_search)   - будут рассматриваться только файлы с данной строкой в названии (регист неважен)
4) Строку (STRING_search) - поиск строки в данном файле (регист неважен) (в случае отсутствия программа просто выдаст все файлы, подходящие под NAME_search)

output:
1) Таблица со столбацами: Путь к файлу, Формат файла (страница при наличии), символы вокруг строки для понимания контекста

Комментарии:
Предполагается, что все функции лежат в отдельном файле.py, но для более удобного чтения решил всё сделать в Jyputer чтобы не отправлять задание отдельной папкой.
А также тут удобнее написать это введение.
Для работы с zip арихивами использовал zipfile, т.к. с большими архивами pyzipper работает медленее, думаю в данном случае это не очень важно и поменять библиотеку не составит особого труда.
Программа сделана таким образом, чтобы легко было добавлять новые форматы файлов в виде новых функций в блок jupyter

Алггоритм программы:
1. Перебираем файлы по названию, находим в названии нужную строку =>
2. Проверяем тип файла, находим подходящий =>
3. Отправляем на чтение, нашли нужные строки в файле =>
4. Отметили файл и контекст в таблице

In [1]:
import zipfile
#import pyzipper
import io
import pandas as pd

from PyPDF2 import PdfReader
from docx import Document

In [2]:
# Путь к ZIP архиву
zip_file_path = 'Архив.zip'
# Пароль от архива
password_to_zip = ''


# Поиск информации в файлах содержащих эту строку в названии
NAME_search = 'Название файла'
# Наличие информации в файле содержащих эту строку внутри содержания файла
STRING_search = 'Строка в файле'

Блок снизу состаит из функций, каждая из которых анализирует 1 тип файлов и возвращает в случае находки целевой информации: Файл, Формат + страницу, контекст нашей строки (для более легкого поиска).

In [3]:
def search_in_PDF(zip_file, pdf_path, search_string):
    with zip_file.open(pdf_path) as pdf_file:
        cont = ''
        pages = ''
        # Создаем объект PdfReader
        reader = PdfReader(io.BytesIO(pdf_file.read()))
        # Перебираем страницы и ищем строку в содержимом
        for page_number, page in enumerate(reader.pages, start=1):
            try:
                text = page.extract_text().lower()
            except:
                print(f"Неудалось прочитать файл {pdf_path}")
                return [0, 0, 0]
            if search_string in text:
                start_index = text.find(search_string)
                start_output = max(0, start_index - 20)
                end_output = min(len(text), start_index + len(search_string) + 30)
                context = text[start_output:end_output]
                pages += f" {page_number}"
                cont  += f"{context}"
                
        if len(pages) == 0:
            return [0, 0, 0]
        else:
            return ([pdf_path, pages, cont])


def search_in_TXT(zip_file, txt_path, search_string):
        cont = ''
        with zip_file.open(txt_path) as txt_file:
            try:
                text = txt_file.read().decode('utf-8').lower()  # Читаем содержимое файла
            except:
                print(f"Неудалось прочитать файл {txt_path}")
                return False
            index = text.find(search_string)  # Ищем строку

            if index != -1:
                start_index = max(0, index - 20)  # Начало вывода (не меньше 0)
                end_index = index + len(search_string) + 30  # Конец вывода
                context = text[start_index:end_index]  # Извлекаем нужный фрагмент
                cont += context

        if len(cont) == 0:
            return False
        else:
            return ([txt_path, cont])


def search_in_DOCX(zip_file, docx_path, search_string):
    cont = ''
    with zip_file.open(docx_path) as docx_file:
        try:
            doc = Document(io.BytesIO(docx_file.read()))
        except:
            print(f"Неудалось прочитать файл {docx_path}")
            return False
        # doc = Document(docx_path)  # Открываем документ
        full_text = []
        
        # Извлекаем текст из всех параграфов
        for para in doc.paragraphs:
            full_text.append(para.text)
        

        # Объединяем текст в одну строку
        text = '\n'.join(full_text).lower()
        index = text.find(search_string)  # Ищем строку

        if index != -1:
            start_index = max(0, index - 20)  # Начало вывода (не меньше 0)
            end_index = index + len(search_string) + 30  # Конец вывода
            context = text[start_index:end_index]  # Извлекаем нужный фрагмент
            cont += context
    
    if len(cont) == 0:
        return False
    else:
        return ([docx_path, cont])

Блок снизу переберает по очереди все файлы в архиве, отправляя на обрабоку только те, которые соответствуют требованиям по названию или формату
Затем выдёт нам таблицу (ну и строчку там где таблица сохраняется в файл я закоментировал, чтобы запустить пару раз, проверить в обычном выводе что всё работает)

In [ ]:
NAME_search = NAME_search.lower()
STRING_search = STRING_search.lower()

#Найденные совпадения выводим в виде таблице: путь к файлу, формат (страница), ближашие символы для более угобного анализа
df = pd.DataFrame(columns=['Путь/название', 'Формат, стр.', 'Контекст'])

def open_zip(zip_path, df, password=''):
    # Открываем ZIP архив
    with zipfile.ZipFile(zip_path, 'r') as zip_file:
        zip_file.setpassword(password)
    
    # Перебираем все файлы в архиве
        for file_name in zip_file.namelist():
            if file_name.endswith('.pdf'):              # Проверяем, является ли файл PDF
                if NAME_search in file_name.lower():    # Проверяем, содержится ли строка в названии файла
                    if STRING_search == '': #Если строку внетри файла искать не надо, то сразу добавляем файл в список
                        out_of_search = 3
                    else:
                        out_of_search = search_in_PDF(zip_file, file_name, STRING_search) # отправляем файл на проверку на наличие совпадений строки в содержимом
                        if len(out_of_search) != [0, 0, 0]:
                            out_of_search = 2

                    #Добавляем найденную информацию в таблицу
                    if out_of_search != 3 and out_of_search != 2:
                        out_of_search = pd.DataFrame({'Путь/название': [out_of_search[0]], 'Формат, стр.': [f"PDF, {out_of_search[1]}"], 'Контекст': [out_of_search[2]]})
                        df = pd.concat([df, out_of_search], ignore_index=True)
                        out_of_search = False
                    elif out_of_search == 3:
                        out_of_search = pd.DataFrame({'Путь/название': [NAME_search], 'Формат, стр.': [f"PDF"], 'Контекст': [" "]})
                    else:
                        pass #тут можно добавить, что информация не найдена
            
            
            elif file_name.endswith('.txt'):            # Проверяем, является ли файл TXT
                if NAME_search in file_name.lower():    # Проверяем, содержится ли строка в названии файла
                    if STRING_search == '':
                        out_of_search = 3
                    else:
                        out_of_search = search_in_TXT(zip_file, file_name, STRING_search) # отправляем файл на проверку на наличие совпадений строки в содержимом
                        if out_of_search == False:
                            out_of_search = 2

                    #Добавляем найденную информацию в таблицу
                    if out_of_search != 3 and out_of_search != 2:
                        out_of_search = pd.DataFrame({'Путь/название': [out_of_search[0]], 'Формат, стр.': [f"TXT"], 'Контекст': [out_of_search[1]]})
                        df = pd.concat([df, out_of_search], ignore_index=True)
                        out_of_search = False
                    elif out_of_search == 3:
                        df = pd.concat([df, pd.DataFrame({'Путь/название': [NAME_search], 'Формат, стр.': [f"TXT"],  'Контекст': [""]})], ignore_index=True)


            elif file_name.endswith('.docx'):           # Проверяем, является ли файл DOCX
                if NAME_search in file_name.lower():    # Проверяем, содержится ли строка в названии файла
                    if STRING_search == '': 
                        out_of_search = 3
                    else:
                        out_of_search = search_in_DOCX(zip_file, file_name, STRING_search) # отправляем файл на проверку на наличие совпадений строки в содержимом
                        if out_of_search == False:
                            out_of_search = 2

                    # #Добавляем найденную информацию в таблицу
                    if out_of_search != 3 and out_of_search != 2:
                        out_of_search = pd.DataFrame({'Путь/название': [out_of_search[0]], 'Формат, стр.': [f"DOCX"], 'Контекст': [out_of_search[1]]})
                        df = pd.concat([df, out_of_search], ignore_index=True)
                        out_of_search = False
                    elif (out_of_search == 3):
                        out_of_search = pd.DataFrame({'Путь/название': [NAME_search], 'Формат, стр.': [f"DOCX"], 'Контекст': [""]})
            

            else:
                pass #Формат файла не поддерживается
    return df
df = open_zip(zip_file_path, df, password_to_zip)

#df.to_csv('output.csv', index=False)

df